In [1]:
import time
import pyspark
from pyspark import SparkContext
from pyspark.storagelevel import StorageLevel

In [2]:
class Power:
    def __init__(self, p):
        self.p = p
        time.sleep(2)
        
    def applyPower(self, x):
        return x**self.p
        
# map
def power_map(num):
    # apply num ** 5
    c = Power(5)
    return c.applyPower(num)

In [3]:
conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')
numbers = sc.textFile("../Data/numbers.txt", 5).map(lambda x : int(x))

21/12/03 10:49:40 WARN Utils: Your hostname, Fans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.1.143.47 instead (on interface en0)
21/12/03 10:49:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/03 10:49:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/03 10:49:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/03 10:49:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/12/03 10:49:42 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [4]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Without Persisting or Caching

In [5]:
start = time.time()
powered_num = numbers.map(power_map)
powered_num.collect()
print("first ", time.time() - start)

start = time.time()
powered_num.collect()
print("second ", time.time() - start)

first  44.19709610939026


second  42.27133393287659


## With Persisting/Caching

In [6]:
powered_num.cache()

PythonRDD[2] at collect at /var/folders/hz/5yfy3bjj1ts_fk0bmfk7xj080000gn/T/ipykernel_22153/705686008.py:3

In [7]:
start = time.time()
powered_num.collect()
print("first ", time.time() - start)  # trigger re-evaluation

first  42.27375912666321


In [8]:
start = time.time()
powered_num.collect()
print("second ", time.time() - start)  # doesn't trigger re-evaluation

second  0.267841100692749


## What happens?

In [9]:
powered_num.persist(StorageLevel.MEMORY_AND_DISK)

Py4JJavaError: An error occurred while calling o36.persist.
: java.lang.UnsupportedOperationException: Cannot change storage level of an RDD after it was already assigned a level
	at org.apache.spark.rdd.RDD.persist(RDD.scala:176)
	at org.apache.spark.rdd.RDD.persist(RDD.scala:200)
	at org.apache.spark.api.java.JavaRDD.persist(JavaRDD.scala:51)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(Client

## How to solve it?

In [10]:
powered_num.unpersist()

PythonRDD[3] at collect at /var/folders/0q/77jvn_416ybbg7m9_c4f7xkw2wwy7f/T/ipykernel_40427/705686008.py:3

## Reading/writing from disk

In [11]:
powered_num.persist(StorageLevel.DISK_ONLY)
start = time.time()
powered_num.collect()
print("first ", time.time() - start)  # trigger reevaluation

start = time.time()
powered_num.collect()
print("second ", time.time() - start)  # doesn't trigger reevaluation

first  42.06355309486389
second  0.0256350040435791


In [12]:
powered_num.unpersist()

PythonRDD[3] at collect at /var/folders/0q/77jvn_416ybbg7m9_c4f7xkw2wwy7f/T/ipykernel_40427/705686008.py:3

In [9]:
sc.stop()